In [1]:
import pandas as pd
import numpy as np
import joblib
import sklearn
#import ensemble
from sklearn import ensemble
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,mean_squared_error,balanced_accuracy_score,recall_score
import random
import lightgbm as lgb

In [2]:
cols =['customer_digital_activity_04',
       'customer_spend_01', 'customer_industry_spend_01',
       'customer_industry_spend_02', 'customer_industry_spend_03',
       'customer_industry_spend_04', 'customer_industry_spend_05',
       'customer_spend_02', 'customer_spend_03', 'customer_merchant_02',
       'customer_merchant_01', 'customer_spend_04', 'customer_spend_05',
       'customer_spend_06', 'customer_spend_07', 'merchant_spend_01',
       'merchant_spend_02', 'merchant_spend_03', 'merchant_spend_04',
       'merchant_spend_05', 'merchant_spend_06', 'merchant_spend_07',
       'merchant_spend_08', 'merchant_profile_01', 'customer_merchant_03',
       'customer_profile_01', 'customer_profile_02',
       'customer_digital_activity_05', 'customer_spend_13',
       'customer_digital_activity_06', 'customer_spend_14',
       'customer_digital_activity_07', 'customer_digital_activity_08',
       'customer_digital_activity_09', 'customer_digital_activity_10',
       'customer_digital_activity_11', 'customer_digital_activity_12',
       'customer_digital_activity_13', 'customer_digital_activity_14',
       'customer_digital_activity_15', 'customer_spend_15',
       'customer_digital_activity_16', 'customer_spend_16',
       'customer_spend_17', 'customer_digital_activity_17',
       'customer_digital_activity_03', 'merchant_spend_11',
       'customer_digital_activity_18', 'customer_digital_activity_19',
       'distance_01', 'customer_digital_activity_20', 'distance_02',
       'distance_03', 'customer_spend_18', 'customer_spend_19',
       'customer_digital_activity_21', 'customer_digital_activity_22',
       'distance_04', 'merchant_profile_02', 'merchant_spend_09',
       'merchant_profile_03', 'customer_digital_activity_01',
       'merchant_spend_10', 'customer_profile_03',
       'customer_digital_activity_02', 'customer_profile_04', 'distance_05','customer', 'merchant']

In [3]:
evaldata = pd.read_parquet('Data/Evaluation/Amex_Campus_Challenge_Eval_Round1.parquet')

In [4]:
evaldata = evaldata[cols]

In [5]:
evaldata

,customer_digital_activity_04,customer_spend_01,customer_industry_spend_01,customer_industry_spend_02,customer_industry_spend_03,customer_industry_spend_04,customer_industry_spend_05,customer_spend_02,customer_spend_03,customer_merchant_02,...,merchant_spend_09,merchant_profile_03,customer_digital_activity_01,merchant_spend_10,customer_profile_03,customer_digital_activity_02,customer_profile_04,distance_05,customer,merchant
0,NaN,112.334000,80.552500,9.0,966.63,12.0,10.0,4.0,41.0,NaN,...,26299.0,4777.0,0.800000,33.300,72.268283,7.000000,423.0,1.621171,467915,599167
1,NaN,112.334000,NaN,NaN,NaN,NaN,NaN,4.0,41.0,NaN,...,7122.0,4803.0,0.800000,793.290,72.268283,7.000000,423.0,2.441944,467915,686617
2,NaN,112.334000,71.192500,3.0,284.77,4.0,4.0,4.0,41.0,NaN,...,7222.0,14860.0,0.800000,100.000,72.268283,7.000000,423.0,2.438082,467915,829193
3,NaN,112.334000,NaN,NaN,NaN,NaN,NaN,4.0,41.0,NaN,...,11410.0,11968.0,0.800000,252.380,72.268283,7.000000,423.0,2.072182,467915,1077034
4,NaN,302.792500,NaN,NaN,NaN,NaN,NaN,3.0,37.0,NaN,...,1847.0,5842.0,0.800000,87.500,72.268283,7.000000,423.0,2.380853,467915,876647
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8496461,NaN,65.181875,171.463500,8.0,3429.27,20.0,16.0,21.0,94.0,NaN,...,25017.0,29209.0,0.000000,49.500,92.116548,1.666667,270.0,22.197181,464447,644485
8496462,NaN,15.680000,11.095556,5.0,99.86,9.0,8.0,2.0,14.0,NaN,...,39731.0,3529.0,0.733333,32.440,34.500012,0.166667,24.0,17.812486,522564,802309
8496463,NaN,61.723409,NaN,NaN,NaN,NaN,NaN,11.0,112.0,NaN,...,2994.0,7534.0,1.000000,198.500,50.484705,0.166667,390.0,8.299482,465030,741321
8496464,NaN,270.908125,31.177692,4.0,405.31,13.0,6.0,10.0,28.0,NaN,...,423.0,241.0,0.916667,33.000,86.240872,0.333333,310.0,4.665418,495259,625845


In [6]:
name='sidmethodwithform1'

In [7]:
rec = joblib.load('submissions/'+name+'/models/rec.joblib')
act = joblib.load('submissions/'+name+'/models/act.joblib')

In [8]:
rec_list = rec.predict_proba(evaldata.drop(['customer','merchant'],axis=1))[:,1]
print('done with first')
act_list = act.predict_proba(evaldata.drop(['customer','merchant'],axis=1))[:,1]

In [18]:
evaldata['rec'] = list(rec_list)
evaldata['act'] = list(act_list)
evaldata['predicted_score'] = (evaldata['rec'].round() - 0.5)*evaldata['act']

In [19]:
evaldata.predicted_score.isnull().any()

False

In [20]:
evaldata.dtypes

customer_digital_activity_04    float64
customer_spend_01               float64
customer_industry_spend_01      float64
customer_industry_spend_02      float64
customer_industry_spend_03      float64
                                 ...   
customer                          int64
merchant                          int64
rec                             float64
act                             float64
predicted_score                 float64
Length: 72, dtype: object

In [21]:
evaldata['predicted_score'].mean()

-0.011063466852327931

In [14]:
evaldata['predicted_score'] = evaldata['predicted_score'].astype('float16')

In [22]:
evaldata[['customer','predicted_score','merchant']].to_csv('submissions/'+name+'/submission.csv',index=False)